###Open file

In [1]:
import glob
import os
import csv
import zipfile
import StringIO


s_name = "/Users/ucaiado/Dropbox/NEUTRINO/ALGO/DATA/pisa2012.csv.zip"

# for name in glob.glob('C:/Projects/abase/*.zip'):
#     base = os.path.basename(name)
#     filename = os.path.splitext(base)[0]


#     datadirectory = 'C:/Projects/abase/'
#     dataFile = filename
#     archive = '.'.join([dataFile, 'zip'])
#     fullpath = ''.join([datadirectory, archive])
#     csv_file = '.'.join([dataFile, 'csv']) #all fixed


#     filehandle = open(fullpath, 'rb')
#     zfile = zipfile.ZipFile(filehandle)
#     data = StringIO.StringIO(zfile.read(csv_file)) #don't forget this line!
#     reader = csv.reader(data)

#     for row in reader:
#         print row

###Create a function to read the file

In [55]:
import csv
import zipfile

def foo(s_name):
    with open(s_name, 'rb') as f1:
        zfile = zipfile.ZipFile(f1)
        with zfile.open(zfile.filelist[0].filename) as f:
            reader = csv.DictReader(f, ",")
            for idx, row in enumerate(reader):
                if idx%50000==0: print "estou na linha {}".format(idx)
        print "estou na linha {}".format(idx)
        
    return row    

In [56]:
%time row = foo(s_name)

estou na linha 0
estou na linha 50000
estou na linha 100000
estou na linha 150000
estou na linha 200000
estou na linha 250000
estou na linha 300000
estou na linha 350000
estou na linha 400000
estou na linha 450000
estou na linha 485490
CPU times: user 1min 45s, sys: 397 ms, total: 1min 45s
Wall time: 1min 45s


###Look at what there are inside the data

In [57]:
import pprint

In [61]:
pprint.pprint(row.keys())

[None, ',']


In [90]:
with open(s_name, 'rb') as f1:
    zfile = zipfile.ZipFile(f1)
    with zfile.open(zfile.filelist[0].filename) as f:
        reader = csv.DictReader(f)
        for idx, row in enumerate(reader):
            if idx==0:break
    print "estou na linha {}".format(idx)

estou na linha 0


In [91]:
len(row)

636

In [97]:
l_schoolSupport = ["DISCLIMA", "MTSUP", "TEACHSUP", "STUDREL", "TCHBEHFA","TCHBEHSO", "TCHBEHTD"]
l_homeSupport = ["HISCED", "HISEI", "HOMEPOS", "CULTPOS", "ESCS", "HEDRES"]


d_data = {k: row.get(k, None) for k in l_schoolSupport + l_homeSupport + ["PV1MATH"]}

In [98]:
d_data

{'CULTPOS': '-0.48',
 'DISCLIMA': '1.85',
 'ESCS': 'NA',
 'HEDRES': '-1.29',
 'HISCED': 'ISCED 3A, ISCED 4',
 'HISEI': 'NA',
 'HOMEPOS': '-2.61',
 'MTSUP': '-0.9508',
 'PV1MATH': '406.8469',
 'STUDREL': '-1.04',
 'TCHBEHFA': '1.3625',
 'TCHBEHSO': '0.9374',
 'TCHBEHTD': '0.4297',
 'TEACHSUP': '1.68'}

In [92]:
pprint.pprint(row)

{'': '1',
 'AGE': '16.17',
 'ANCATSCHL': '-1.8636',
 'ANCATTLNACT': '-0.6779',
 'ANCBELONG': '-0.7351',
 'ANCCLSMAN': '-0.7808',
 'ANCCOGACT': '-0.0219',
 'ANCINSTMOT': '-0.1562',
 'ANCINTMAT': '0.0486',
 'ANCMATWKETH': '-0.2199',
 'ANCMTSUP': '-0.5983',
 'ANCSCMAT': '-0.0807',
 'ANCSTUDREL': '-0.5901',
 'ANCSUBNORM': '-0.3346',
 'ANXMAT': '0.32',
 'ATSCHL': '-2.31',
 'ATTLNACT': '0.5206',
 'BELONG': '-1.18',
 'BFMJ2': '76.49',
 'BMMJ1': '79.74',
 'BOOKID': 'booklet 7',
 'CLCUSE1': 'A Simple calculator',
 'CLCUSE301': '99',
 'CLCUSE302': '99',
 'CLSMAN': '-1.3771',
 'CNT': 'Albania',
 'COBN_F': 'Albania',
 'COBN_M': 'Albania',
 'COBN_S': 'Albania',
 'COGACT': '0.6994',
 'CULTDIST': 'NA',
 'CULTPOS': '-0.48',
 'DEFFORT': '99',
 'DISCLIMA': '1.85',
 'EASY': 'Standard set of booklets',
 'EC01Q01': 'NA',
 'EC02Q01': 'NA',
 'EC03Q01': 'NA',
 'EC03Q02': 'NA',
 'EC03Q03': 'NA',
 'EC03Q04': 'NA',
 'EC03Q05': 'NA',
 'EC03Q06': 'NA',
 'EC03Q07': 'NA',
 'EC03Q08': 'NA',
 'EC03Q09': 'NA',
 'EC03Q1